In [1]:
from Dataload import dataload

from module import transformer,video_model,invres

import torch
import torch.nn as nn
from torchsummary import summary
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
import matplotlib.pylab as plt
from torchvision import  utils
import os
from constant import EMOTIPATH
import time
from tqdm.notebook import tqdm


In [2]:
Train_label=os.path.join(EMOTIPATH,"Train_labels.txt")
Train_video=os.path.join(EMOTIPATH,"Train")
Val_labels=os.path.join(EMOTIPATH,"Val_labels.txt")
Val_video=os.path.join(EMOTIPATH,"Val")

frame_num=1
train_data=dataload.Video_Frame_Data(Train_label,"train_new.csv",base_path_v=Train_video,frame_num=frame_num)



In [5]:
train_data[0][0].shape

torch.Size([1, 1000])

In [6]:
#Batchfy
#Counts number of correct label
def num_correct(prediction,labels):
    correct=0
    for i,(pred_label,label) in enumerate(zip(prediction,labels)):
        if (pred_label.item()==label.item()):
            correct +=1
    return correct


valid_data=dataload.Video_Frame_Data(Val_labels,"train_new.csv",base_path_v=Val_video,frame_num=frame_num)

#train_data1=dataload.Video_Embedding_Data("train.csv",Train_label)
#valid_data=dataload.Video_Embedding_Data("valid.csv",Val_labels)
            
train_dataloader = DataLoader(train_data, batch_size=32
                       , num_workers=0,shuffle=True)
#valid_data=dataload.Video_Frame_Data(Val_labels,base_path_v=Val_video,strict_num=26)

valid_dataloader = DataLoader(valid_data, batch_size=32
                       , num_workers=0)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.cuda.FloatTensor)
model=video_model.Encoder(num_frame=frame_num,isframe=1)
for name, child in model.named_children():
   if not name in ['densenet']:
       print(name + ' is unfrozen')
       for param in child.parameters():
           param.requires_grad = True
   else:
       print(name + ' is frozen')
       for param in child.parameters():
           param.requires_grad = False
#model=model.to(device)  
model=model.double().to(device)
#model=model.float()
criterion = nn.CrossEntropyLoss()  
loss_func = torch.nn.MSELoss()

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0002, betas=(0.5, 0.999))

len(train_data)

embedder is unfrozen
fc1 is unfrozen
fc2 is unfrozen
posencoding is unfrozen
transformer is unfrozen


2661

In [7]:
def train(num_epochs,name,model):
    f = open(name+".txt",'a')
    start = time.time()
    #Triaining
    train_loss=[]
    valid_accuracy=[]
    model.train()
    for epochs in range(0,num_epochs):
        first=True
        model.train()
        correct=0
        total_samples=0
        avg_tloss=0
        print("Training Epoch: ", epochs+1,"\n")
        for i_batch, (sample_batched,_,label) in tqdm(enumerate(train_dataloader)):
           # sample_batched=sample_batched.frames
            batch_size=sample_batched.size(0)
            optimizer.zero_grad()
            sample_batched=sample_batched.to(device)
            output=model(sample_batched)
            loss=criterion(output,label.to(device))
            loss.backward()
            predicted = torch.max(output, 1)
            prediction=predicted.indices.detach().cpu()
            correct +=num_correct(prediction,label)
            total_samples+=batch_size
            accuracy=correct/(total_samples)
            optimizer.step()
            true_label=label.detach().cpu()
            avg_tloss+=loss.item()
            if first:
                first=False
                conf_mat=confusion_matrix( true_label,prediction,labels=[0,1,2])
            else:
                conf_mat+=confusion_matrix(true_label,prediction,labels=[0,1,2])
            if i_batch%20==0:
               # print(label)
                print("Batch: ",i_batch+1,"/",len(train_dataloader))
                print("Batch Recognition loss: ", loss.item())

        print(conf_mat)
        avg_tloss=avg_tloss/len(train_dataloader)
        avg_taccuracy=correct/total_samples
        print("Average_Loss: ",avg_tloss)
        print("Average_Accuracy: ",avg_taccuracy)

        torch.save(model.state_dict(),name+".pth")
        print("Validation\n")
        model.eval()   
        correct=0
        total_samples=0
        avg_vloss=0
        first=True

        for i_batch, (sample_batched,_,label) in enumerate(valid_dataloader):
            batch_size=sample_batched.size(0)
            sample_batched=sample_batched.to(device)
            output=model(sample_batched)
            loss=criterion(output,label.to(device))
            avg_vloss+=loss.item()
            predicted = torch.max(output, 1)
            prediction=predicted.indices.detach().cpu()
            correct +=num_correct(prediction,label)
            total_samples+=batch_size

            true_label=label.detach().cpu()
            if first:
                first=False
                conf_mat=confusion_matrix( true_label,prediction,labels=[0,1,2])
            else:
                conf_mat+=confusion_matrix(true_label,prediction,labels=[0,1,2])
        print(conf_mat)
        avg_vloss=avg_vloss/len(valid_dataloader)
        print(avg_vloss)
        avg_vaccuracy=correct/(total_samples)
        print("Accuracy: ", avg_vaccuracy)

        data = " %f,%f,%f,%f \n" % (avg_tloss,avg_taccuracy,avg_vloss,avg_vaccuracy)
        f.write(data)



    f.close()


In [8]:
model_name="pre_embedded_1_frames"
train(20,model_name,model)

Training Epoch:  1 



Batch:  1 / 84
Batch Recognition loss:  1.102019359660783
Batch:  21 / 84
Batch Recognition loss:  1.0067424240337943
Batch:  41 / 84
Batch Recognition loss:  0.8891175967320328
Batch:  61 / 84
Batch Recognition loss:  1.084498578127521
Batch:  81 / 84
Batch Recognition loss:  1.0271528711184057

[[255 278 273]
 [174 476 273]
 [115 189 628]]
Average_Loss:  1.010583182093453
Average_Accuracy:  0.5107102593010147
Validation

[[158  68  73]
 [188  64  29]
 [154  17  15]]
1.1787222578954624
Accuracy:  0.3093994778067885
Training Epoch:  2 



Batch:  1 / 84
Batch Recognition loss:  0.9630548682869428
Batch:  21 / 84
Batch Recognition loss:  0.9699716250431251
Batch:  41 / 84
Batch Recognition loss:  0.943315333444924
Batch:  61 / 84
Batch Recognition loss:  0.9354708185384353
Batch:  81 / 84
Batch Recognition loss:  0.9208720461929907

[[390 223 193]
 [229 481 213]
 [113 132 687]]
Average_Loss:  0.9489865723194912
Average_Accuracy:  0.5854941751221345
Validation

[[ 74 218   7]
 [ 75 201   5]
 [ 58 123   5]]
1.1353248146008197
Accuracy:  0.36553524804177545
Training Epoch:  3 



Batch:  1 / 84
Batch Recognition loss:  0.9580147096886342
Batch:  21 / 84
Batch Recognition loss:  0.8499259133099203
Batch:  41 / 84
Batch Recognition loss:  0.9296086452449996
Batch:  61 / 84
Batch Recognition loss:  0.9235689916120381
Batch:  81 / 84
Batch Recognition loss:  0.8529874102105642

[[441 205 160]
 [191 543 189]
 [106 143 683]]
Average_Loss:  0.9170601550995069
Average_Accuracy:  0.626456219466366
Validation

[[108 144  47]
 [109 160  12]
 [ 83  95   8]]
1.153292243618014
Accuracy:  0.360313315926893
Training Epoch:  4 



Batch:  1 / 84
Batch Recognition loss:  0.8307077192832759
Batch:  21 / 84
Batch Recognition loss:  0.884094779300902
Batch:  41 / 84
Batch Recognition loss:  0.9959029580354364
Batch:  61 / 84
Batch Recognition loss:  0.9027526457112484
Batch:  81 / 84
Batch Recognition loss:  0.8338239693460834

[[468 188 150]
 [195 534 194]
 [ 97 133 702]]
Average_Loss:  0.8985577687981801
Average_Accuracy:  0.6403607666290868
Validation

[[123 131  45]
 [126 137  18]
 [ 92  87   7]]
1.153132828978449
Accuracy:  0.34856396866840733
Training Epoch:  5 



...

Batch:  1 / 84
Batch Recognition loss:  0.7112684880786044
Batch:  21 / 84
Batch Recognition loss:  0.8350842836684247
Batch:  41 / 84
Batch Recognition loss:  0.8232997270800467
Batch:  61 / 84
Batch Recognition loss:  0.8136152017928022
Batch:  81 / 84
Batch Recognition loss:  0.859063415796094

[[579 131  96]
 [150 656 117]
 [ 49 100 783]]
Average_Loss:  0.7973656691213006
Average_Accuracy:  0.7583615182262308
Validation

[[108 127  64]
 [ 98 156  27]
 [ 79  93  14]]
1.1693151059427194
Accuracy:  0.3629242819843342
Training Epoch:  17 



Batch:  1 / 84
Batch Recognition loss:  0.8373484170337316
Batch:  21 / 84
Batch Recognition loss:  0.7774899667292885
Batch:  41 / 84
Batch Recognition loss:  0.8716312595260252
Batch:  61 / 84
Batch Recognition loss:  0.7367976135068652
Batch:  81 / 84
Batch Recognition loss:  0.8547476956633675

[[579 132  95]
 [125 671 127]
 [ 48 102 782]]
Average_Loss:  0.7897025164129834
Average_Accuracy:  0.7636226982337467
Validation

[[138 117  44]
 [136 133  12]
 [109  70   7]]
1.1672317785061777
Accuracy:  0.3629242819843342
Training Epoch:  18 



Batch:  1 / 84
Batch Recognition loss:  0.6984263319886894
Batch:  21 / 84
Batch Recognition loss:  0.8186335166896438
Batch:  41 / 84
Batch Recognition loss:  0.8490359118710014
Batch:  61 / 84
Batch Recognition loss:  0.7996109364446272
Batch:  81 / 84
Batch Recognition loss:  0.7963192593666464

[[597 121  88]
 [134 667 122]
 [ 52  95 785]]
Average_Loss:  0.7838567907075222
Average_Accuracy:  0.770011273957159
Validation

[[136  97  66]
 [127 123  31]
 [101  71  14]]
1.1705148228326572
Accuracy:  0.3563968668407311
Training Epoch:  19 



Batch:  1 / 84
Batch Recognition loss:  0.7796689905538109
Batch:  21 / 84
Batch Recognition loss:  0.7220358807107771
Batch:  41 / 84
Batch Recognition loss:  0.777804916979323
Batch:  61 / 84
Batch Recognition loss:  0.7931002050852936
Batch:  81 / 84
Batch Recognition loss:  0.8157377013315962

[[600 117  89]
 [133 661 129]
 [ 50  97 785]]
Average_Loss:  0.7873943366567879
Average_Accuracy:  0.7688838782412627
Validation

[[144 120  35]
 [129 145   7]
 [103  79   4]]
1.147574156259897
Accuracy:  0.3825065274151436
Training Epoch:  20 



Batch:  1 / 84
Batch Recognition loss:  0.7885921683776084
Batch:  21 / 84
Batch Recognition loss:  0.812432181204799
Batch:  41 / 84
Batch Recognition loss:  0.7980728745366531
Batch:  61 / 84
Batch Recognition loss:  0.8278675745058263
Batch:  81 / 84
Batch Recognition loss:  0.7536954537934001

[[607 117  82]
 [114 694 115]
 [ 41  94 797]]
Average_Loss:  0.7733446067830165
Average_Accuracy:  0.7884254039834648
Validation

[[155 110  34]
 [141 133   7]
 [103  77   6]]
1.1471737572203533
Accuracy:  0.3838120104438642


In [ ]:
print("finished")

In [ ]:
valid_data1=dataload.Video_Frame_Data(Val_labels,base_path_v=Val_video,strict_num=26)

valid_dataloader1 = DataLoader(valid_data1, batch_size=32
                       , num_workers=0)

In [ ]:
debug

In [ ]:
train(1)